# Obteniendo datos de la API de Spotify

In [18]:
import base64
import requests
import pandas as pd
import os
import discogs_client
import time

## Autorización

In [2]:
client_id = os.environ['spotify_client_id']
client_secret = os.environ['spotify_client_secret']

In [3]:
# función de autenticación de spotify

def spoty_aut (client_id, client_secret):
    url = 'https://accounts.spotify.com/api/token'
    credenciales = client_id + ':' + client_secret
    authorization = "Basic " + base64.b64encode(credenciales.encode()).decode()
    grant_type= 'client_credentials'
    auth = requests.post(url, data={'grant_type':grant_type},
              headers={'Authorization':authorization, 'Content-Type':'application/x-www-form-urlencoded'})
    return auth.json()

In [4]:
auth=spoty_aut(client_id, client_secret)

In [5]:
auth

{'access_token': 'BQAQvyj_nNa7XOTalYSTiEAcHaQw1WrzdWC_KE9gM3sge-vggqtaLeDgwMq9u35WJ2tsQFmQOsqwIUGoM7I',
 'token_type': 'Bearer',
 'expires_in': 3600}

In [6]:
# autorización de la API de discogs
d = discogs_client.Client('ExampleApplication/0.1', user_token="FegwZkxqEcDNITLmmVYWuQNQZAMdnpBWeVTWZhts")

## Obteniendo playlist

In [7]:
    base = "https://api.spotify.com/v1/"
    playlist = requests.get(base+f"playlists/6doANzWu81vKWBsIiMLrHj/tracks",
                      headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    

In [8]:
playlist.json()['items'][0]['track']['album']['name']

'Computer Controlled Acoustic Instruments pt2 EP'

In [9]:
#función que crea df con variables necesarias para sherpa a partir de playlist_id

def df_sherpa(playlist_id):
    base = "https://api.spotify.com/v1/"
    playlist = requests.get(base+f"playlists/{playlist_id}/tracks",
                      headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    
    n_canciones=len(playlist.json()['items'])
    
    lista_df_tracks=[]
    
    for i in range(0,n_canciones):
        
        #obteniendo autor, nombre de canción y sus IDs de las canciones en la playlist
        artist_track=playlist.json()['items'][i]
        
        artist_id= artist_track['track']['artists'][0]['id']
        artist = artist_track['track']['artists'][0]['name']
        track = artist_track['track']['name']
        track_id = artist_track['track']['id']
        album = artist_track['track']['album']['name']
        duration=artist_track['track']['duration_ms']/60_000
        
        #obteniendo características de canciones de playlist
        audio_features = requests.get(base+f"audio-features/{track_id}",
                          headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
        tempo = audio_features.json()['tempo']
        energy = audio_features.json()['energy']
        danceability = audio_features.json()['danceability']
        valence = audio_features.json()['valence']
        key = audio_features.json()['key']
        time_signature = audio_features.json()['time_signature']
        acousticness = audio_features.json()['acousticness']
        instrumentalness = audio_features.json()['instrumentalness']
        mode = audio_features.json()['mode']
    
        lista_track = [track_id, artist, track, album, duration, tempo, energy, danceability, 
                 valence, key, time_signature, acousticness, instrumentalness, mode]
        lista_df_tracks.append(lista_track)
    
    df_sherpa=pd.DataFrame(lista_df_tracks, columns=["track_id", "artist", "track", "album", "duration",  "tempo", "energy", "danceability", 
             "valence", "key", "time_signature", "acousticness", "instrumentalness", "mode"])
    return df_sherpa
    
    

In [10]:
#Aplicando la función para obtener variables de estudio de canciones de una playlist
df_test=df_sherpa("6doANzWu81vKWBsIiMLrHj")

## Discogs database

In [12]:

styles=[]

for track_id in df_test['track_id']:
    base = "https://api.spotify.com/v1/"
    song = requests.get(base+f"tracks/{track_id}",
                      headers = {'Authorization':f'''Bearer {auth['access_token']}''', 'Content-Type': 'application/json'})
    
    album=song.json()['album']['name']
    
     
    results = d.search(f'{album}')
    if len(results)==0:
        styles.append(['Not data', album])
    else:    
        if str(results[0]).startswith('<Label') :
            if str(results[1]).startswith('<Release') :
                release=results[1].id
                styles.append([d.release(release).styles, album])
            elif str(results[1]).startswith('<Master'):
                release=results[1].id
                styles.append([d.master(release).styles, album])
        else:
            if str(results[0]).startswith('<Release') :
                release=results[0].id
                styles.append([d.release(release).styles, album])
            elif str(results[0]).startswith('<Master'):
                release=results[0].id
                styles.append([d.master(release).styles, album])    

In [13]:
styles=pd.DataFrame(styles, columns=['styles', 'album'])

In [15]:
df_test=pd.merge(df_test, styles, on='album', how='left')

In [26]:
dic={0:"C", 1:"C#", 2:"D", 3:"D#", 4:"E", 5:"F", 6:"F#", 7:"G", 8:"G#", 9:"A", 10:"A#", 11:"B"}
df_test=df_test.replace({"key":dic})

In [27]:
df_test.to_csv("/Users/santiago/ih-bootcamp/proyectos/sherpa/out/sherpa_test.csv")

In [39]:
a=[len(cancion) for cancion in df_test.dropna()['styles']]

In [40]:
max(a)

8

In [44]:
df_styles=df_test[['track', 'styles']]

In [76]:
string=' '.join(map(str, df_styles['styles']))
string=string.replace("]", ",").replace('[', "").replace('nan', "").replace('Not data', "")
print(string)


'IDM', 'Experimental', 'Modern', 'Musique Concrète', 'Minimal', 'IDM', 'Hindustani', 'Fusion', 'Afrobeat', 'African', 'Experimental', 'House', 'Techno', 'IDM', 'Experimental', 'House', 'Deep House', 'Disco', 'Deep House', 'Techno', 'Minimal', 'Deep House', 'African', 'Experimental', 'Rockabilly', 'Deep House', 'Modern', 'Avantgarde', 'Experimental', 'Nu-Disco', 'Techno', 'House', 'Indie Rock', 'Deep House', 'Art Rock', 'Ambient', 'Experimental',   'Reggaeton',  'Experimental', 'Ambient', 'Leftfield',          'House',   'Techno', 'Dub Techno', 'House', 'Techno', 'Dub Techno', 'House', 'Techno', 'Dub Techno', 'House', 'Techno', 'Dub Techno', 'House', 'Cumbia', 'Disco', 'Psychedelic', 'Soukous', 'House', 'Highlife',   'Progressive House', 'Techno', 'Progressive Trance',     'Leftfield', 'Downtempo', 'Witch House', 'Dubstep', 'Modern Classical', 'Abstract', 'Ambient', 'Drone',  'Techno', 'Electro', 'Downtempo', 'Experimental', 'IDM', 'Techno', 'Tech House', 'Deep House', 'House', 'Techno'